In [49]:
from sklearn import svm
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import cv2
import os
import numpy as np

In [50]:
preprocessed_images = []
feature_vectors = []

In [51]:
def extract_hog_features(image, cell_size, num_orientations):

  # Calculate the HOG features
  hog = cv2.HOGDescriptor(_winSize=(image.shape[1] // cell_size[1] * cell_size[1],
                                      image.shape[0] // cell_size[0] * cell_size[0]),
                          _blockSize=(cell_size[1] * 2, cell_size[0] * 2),
                          _blockStride=(cell_size[1], cell_size[0]),
                          _cellSize=(cell_size[1], cell_size[0]),
                          _nbins=num_orientations)
  features = hog.compute(image)

  # Return the features
  return features

In [52]:
def hog_preprocessing(image):
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    filtered_image  = cv2.GaussianBlur(grayscale_image, (3, 3), 1.0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    normalized_image = clahe.apply(filtered_image)
    img_array = np.array(normalized_image, dtype=np.float32) / 255.0
    corrected_array = np.power(img_array, 0.5)
    corrected_image = np.uint8(corrected_array * 255.0)
    resized_image = cv2.resize(corrected_image,(1280,1280))
    return resized_image

In [53]:
def load_images_from_folder(gender, number):

    # Set the path to the folder containing the images
    folder_path = '../data/' + gender + '/' + str(number)

    # Create an empty list to store the images
    feature_vectors = []

    # Create an empty list to store the labels
    labels = []

    # Loop through all the images in the folder
    for imagefile in os.listdir(folder_path):
        # Read the image 
        image = cv2.imread(os.path.join(folder_path, imagefile))

        # Convert the color channels from BGR to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        preprocessed_image = hog_preprocessing(image)

        feature_vector = extract_hog_features(preprocessed_image, np.array([8, 8]), 9)

        # Append the image to the list of images
        feature_vectors.append(feature_vector)

        # Append the label to the list of labels
        labels.append(number)

        del image

    return feature_vectors, labels

In [54]:
def load_dataset():
    X = []
    Y = []
    x, y = load_images_from_folder('men', 0)
    X += x
    Y += y
    x, y = load_images_from_folder('Women', 0)
    X += x
    Y += y
    x, y = load_images_from_folder('men', 1)
    X += x
    Y += y
    x, y = load_images_from_folder('Women', 1)
    X += x
    Y += y
    x, y = load_images_from_folder('men', 2)
    X += x
    Y += y
    x, y = load_images_from_folder('Women', 2)
    X += x
    Y += y
    x, y = load_images_from_folder('men', 3)
    X += x
    Y += y
    x, y = load_images_from_folder('Women', 3)
    X += x
    Y += y
    x, y = load_images_from_folder('men', 4)
    X += x
    Y += y
    x, y = load_images_from_folder('Women', 4)
    X += x
    Y += y
    x, y = load_images_from_folder('men', 5)
    X += x
    Y += y
    x, y = load_images_from_folder('Women', 5)
    X += x
    Y += y

    return np.array(X), np.array(Y)

In [67]:
# Load the dataset
X, y = load_dataset()

In [68]:
# Concatenate the arrays element-wise
# Reshape the second array to be a 2D array of size (numberofsamples, 1)
y = y.reshape((y.shape[0], 1))

# open the file in append mode
with open('Label_FeatureVector.csv', 'w') as f:
    f.truncate(0)

# open the file in append mode and write new data to it
with open('Label_FeatureVector.csv', 'a') as f:
    for i in range(0,y.shape[0],23):
        # Save in csv file
        np.savetxt('Label_FeatureVector.csv',  np.concatenate((y[i:i+23], X[i:i+23]), axis=1), delimiter=',')

In [ ]:
# split it into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Create an SVM classifier
clf = svm.SVC(kernel='linear', C=1)

In [ ]:
# Train the SVM classifier on the training data
clf.fit(X_train, y_train)

MemoryError: Unable to allocate 9.87 GiB for an array with shape (1456, 910116) and data type float64

In [ ]:
# Make predictions on the testing data
y_pred = clf.predict(X_test)

In [ ]:
# Calculate the accuracy of the SVM classifier
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
# Print the accuracy of the SVM classifier
print("Accuracy:", accuracy*100, '%')